## Evaluate imputation panels

### prepare datasets

#### multiple imps (UKB)

In [2]:
discovery="PGC2_noAJ"
target="dbg-scz19"
imp_pops="eur,sas,gbr"
imp_pops=(${imp_pops//,/ })
base_rep="105"
n_folds=5
pheno=ht # TBD Probably redundant. Check it!
## Adjust reps and folds according to a parameter

#Prepare inner CV (train+validation)
for imp_pop in imp_pops; do 
    for cur_rep in {1..6}; do 
        for cur_fold in {1..5}; do 
            bash prepare_cv_datasets.sh --discovery ${discovery} --target ${target} \
            --imp impute2_1kg_${imp_pop} --rep ${base_rep}_${cur_rep} \
            --cv ${cur_fold}_${n_folds} --pheno ${pheno}; 
        done 
    done
done

#Prepare outer CV (both+test)
for imp_pop in imp_pops; do 
    for cur_rep in {1..6}; do 
        bash prepare_cv_datasets.sh --discovery ${discovery} --target ${target} \
        --imp impute2_1kg_${imp_pop} --rep ${base_rep}_${cur_rep} \
        --cv ${n_folds} --pheno ${pheno} 
    done 
done


bash: prepare_cv_datasets.sh: No such file or directory
bash: prepare_cv_datasets.sh: No such file or directory
bash: prepare_cv_datasets.sh: No such file or directory
bash: prepare_cv_datasets.sh: No such file or directory
bash: prepare_cv_datasets.sh: No such file or directory
bash: prepare_cv_datasets.sh: No such file or directory
bash: prepare_cv_datasets.sh: No such file or directory
bash: prepare_cv_datasets.sh: No such file or directory
bash: prepare_cv_datasets.sh: No such file or directory
bash: prepare_cv_datasets.sh: No such file or directory
bash: prepare_cv_datasets.sh: No such file or directory
bash: prepare_cv_datasets.sh: No such file or directory
bash: prepare_cv_datasets.sh: No such file or directory
bash: prepare_cv_datasets.sh: No such file or directory
bash: prepare_cv_datasets.sh: No such file or directory
bash: prepare_cv_datasets.sh: No such file or directory
bash: prepare_cv_datasets.sh: No such file or directory
bash: prepare_cv_datasets.sh: No such file or di

: 127

### Generate pheno files to subsets defined for the nested CV (```prepare_datsets.sh```)

In [ ]:

for cur_rep in {1..6}; do 
    cd rep_105_${cur_rep}
    for pheno in chol t2d gerx madd angna ast ctrt gerx height hfvr hyty osar utfi; do 
        echo "rep_105_${cur_rep} ${pheno}"
        ls -1 pheno_ht_* | xargs -I A bash -c 'fl=$(echo A | awk '"'"'{ gsub(/_ht_/, "_'${pheno}'_"); print $0}'"'"'); awk '"'"'{if(NR==FNR){c[$1]++; next}; if (c[$1]>0){print $0}}'"'"' <(cat A) <(cat ../pheno_'${pheno}'_) > ${fl}'
    done
    cd ..
done

### Run CV across folds

#### Generally the best practive it os run the scripts ```run_folds/run_UKB_folds_<method>.sh```. The run only parts that are missing in the CV scheme and is generally faster and safer.

In [ ]:
method=pt3

discoveries="D2_sysp_evangelou_2018,D2_ldlp_willer_2013,D2_t2di_mahajan_2018,D2_gerx_an_2019,D2_madd_howard_2019" # ",D2_dias_evangelou_2018" # "D2_sysp_evangelou_2018,D2_dias_evangelou_2018,D2_asth_zhu_2019,D2_chol_willer_2013,D2_ldlp_willer_2013,D2_t2di_mahajan_2018,D2_gerx_an_2019,D2_madd_howard_2019" # "D2_sysp_evangelou_2018" D2_hdlp_willer_2013
# discoveries="UKB_ht_eur,UKB_chol_eur,UKB_hfvr_eur,UKB_hyty_eur,UKB_madd_eur,UKB_osar_eur,UKB_t2d_eur,UKB_utfi_eur,UKB_gerx_eur,UKB_angna_eur,UKB_ast_eur,UKB_ctrt_eur"
targets="ukbb_afr,ukbb_sas"
imps="original,impute2_1kg_eur,impute2_1kg_afr,impute2_1kg_sas" # "impute2_1kg_gbr" # ,impute2_1kg_afr,impute2_1kg_gbr" # "impute2_1kg_eur,impute2_1kg_afr,impute2_1kg_gbr,impute2_1kg_sas" # "impute2_1kg_ceu2"

bash run_UKB_folds_${method}.sh --discoveries=${discoveries} --targets=${targets} --imps=${imps} --ld=eur

#### To override existing results, use the scripts below:

#### Run PRS method across reps

In [ ]:
discovery="PGC2_noAJ"
target="dbg-scz19"
imps="impute2_ajkg14_t101 impute2_1kg_ceu impute2_1kg_gbr impute2_1kg_ibs impute2_1kg_tsi impute2_1kg_fin" # imp="impute2_ajkg14_t101" # "impute2_1kg_ceu2"
method="ls"
base_rep=105
for imp in $imps; do 
    for cur_rep in {1..6}; do 
        bash run_cv_single_rep.sh --discovery ${discovery} --target ${target} --imp ${imp} \
        --method ${method} --rep ${base_rep}_${cur_rep} --stage 3 && \
        bash run_cv_test_single_rep.sh --discovery ${discovery} --target ${target} --imp ${imp} \
        --method ${method} --rep ${base_rep}_${cur_rep}  --stage 3; 
    done

done 

#### Altentively, it can be executed as two separated commads (usefule for reruns of failed executions)

In [ ]:
discovery="PGC2_noAJ"
target="dbg-scz19"
imp="impute2_ajkg14_t101" # "impute2_1kg_ceu2"
method="ld"
base_rep=105

for cur_rep in {1..6}; do 
    bash run_cv_single_rep.sh --discovery ${discovery} --target ${target} --imp ${imp} \
    --method ${method} --rep ${base_rep}_${cur_rep}
done

for cur_rep in {1..6}; do 
    bash run_cv_test_single_rep.sh --discovery ${discovery} --target ${target} --imp ${imp} \
    --method ${method} --rep ${base_rep}_${cur_rep}
done


### Clean lasso output weight files from 0 weighted SNPs

In [ ]:
cat $PRS_CODEBASE/lasso_folders | head -n 5 | xargs -I {} bash -c 'ls -d {}*' | \
 xargs -I A -n 1 -P 80  bash -c  'echo "start A" && cat A | awk '"'"'{if($6!=0){print $0}}'"'"' > "A_" && sleep 0 | mv A_ A && echo "done A"'

## Revision

#### Evaluate PRS quality using EUR GWASs

In [ ]:
cd $PRS_CODEBASE

method=ls # pt3 pt2
base_rep=105
folds=5
rep_start=1
rep_end=6
fold_start=1
fold_end=5

# phenos="ctrt t2d ast"
# phenos="gerx angna utfi" # utfi # "ctrt t2d ast utfi gerx angna" #  chol hfvr hyty ht osar madd" #  "ctrt t2d ast utfi gerx angna" # ctrt chol
# phenos="chol hfvr hyty"
phenos="ht osar madd"
# phenos="ctrt t2d ast gerx angna utfi chol hfvr hyty ht osar madd"

targets="ukbb_sas,ukbb_afr"
imps="original,impute2_1kg_eur,impute2_1kg_sas,impute2_1kg_afr" 

for pheno in $phenos; do 
    discoveries="UKB_${pheno}_eur"
    bash run_UKB_folds_${method}.sh --discoveries=${discoveries} --targets=${targets} --imps=${imps} --pheno=${pheno} --ld=eur \
                                    --base_rep=${base_rep} --folds=${folds} --rep_start=${rep_start} --rep_end=${rep_end} --fold_start=${fold_start} --fold_end=${fold_end} & 
done

#### Evalute PRS quality using LDpred2 PRS method (`run_UKB_folds_ld.sh`). Note that it due to computational considerations, we use do only 3X2 nested-CV  

In [ ]:
cd $PRS_CODEBASE

method=ld
base_rep=102
folds=2
rep_start=1
rep_end=3
fold_start=1
fold_end=2

# phenos="ctrt t2d ast"
# phenos="gerx angna utfi" # utfi # "ctrt t2d ast utfi gerx angna" #  chol hfvr hyty ht osar madd" #  "ctrt t2d ast utfi gerx angna" # ctrt chol
# phenos="chol hfvr hyty"
phenos="ht osar madd"
# phenos="ctrt t2d ast gerx angna utfi chol hfvr hyty ht osar madd"

targets="ukbb_sas,ukbb_afr"
imps="original,impute2_1kg_eur,impute2_1kg_sas,impute2_1kg_afr"
    
for pheno in $phenos; do 
    discoveries="UKB_${pheno}_eur"
    bash run_UKB_folds_${method}.sh --discoveries=${discoveries} --targets=${targets} --imps=${imps} --pheno=${pheno} --ld=eur \
                                    --base_rep=${base_rep} --folds=${folds} --rep_start=${rep_start} --rep_end=${rep_end} --fold_start=${fold_start} --fold_end=${fold_end} & 
done

#### Public EUR GWASs

In [ ]:
method=pt3
base_rep=105
folds=5
rep_start=1
rep_end=6
fold_start=1
fold_end=5

# discoveries="D2_sysp_evangelou_2018 D2_ldlp_willer_2013 D2_t2di_mahajan_2018 D2_gerx_an_2019 D2_madd_howard_2019" 

discoveries="D2_sysp_evangelou_2018" 
# discoveries="D2_ldlp_willer_2013" 
# discoveries="D2_t2di_mahajan_2018" 
# discoveries="D2_gerx_an_2019" 
# discoveries="D2_madd_howard_2019" 

imps="impute2_1kg_afr" # "original,impute2_1kg_eur,impute2_1kg_sas,impute2_1kg_afr" 
targets="ukbb_afr" 

for discovery in $discoveries; do 
    bash run_UKB_folds_${method}.sh --discoveries=${discovery} --targets=${targets} --imps=${imps} --ld=eur \
                                    --base_rep=${base_rep} --folds=${folds} --rep_start=${rep_start} --rep_end=${rep_end} --fold_start=${fold_start} --fold_end=${fold_end}
done


#### EUR subpopulation panel analysis  

In [ ]:
method=ld # ls pt2 pt3
base_rep=105
folds=5
rep_start=1
rep_end=6
fold_start=1
fold_end=5

imps="impute2_ajkg14_t101 impute2_1kg_ceu2 impute2_1kg_gbr2" 
# imps="impute2_1kg_ibs2 impute2_1kg_tsi2 impute2_1kg_fin2" # imp="impute2_ajkg14_t101" # "impute2_1kg_ceu2"

discoveries="PGC2_noAJ"
targets="dbg-scz19" 

for imp in $imps; do 
    bash run_UKB_folds_${method}.sh --discoveries=${discoveries} --targets=${targets} --imps=${imp} --ld=eur \
                                    --base_rep=${base_rep} --folds=${folds} --rep_start=${rep_start} --rep_end=${rep_end} --fold_start=${fold_start} --fold_end=${fold_end}
done


#### Evalute PRS quality using 100-size EUR imputation panel (`run_UKB_folds_{method}.sh`)

In [ ]:
method=ls # pt3 pt2
base_rep=105
folds=5
rep_start=1
rep_end=6
fold_start=1
fold_end=5

pheno=ctrt # ctrt chol hfvr hyty ht osar madd t2d ast utfi gerx angna 

targets="ukbb_sas" # "ukbb_sas,ukbb_afr"
imps="impute2_1kg_eur100"

for pheno in $phenos; do 
    discoveries="UKB_${pheno}_eur"
    bash run_UKB_folds_${method}.sh --discoveries=${discoveries} --targets=${targets} --imps=${imps} --pheno=${pheno} --ld=eur \
                                    --base_rep=${base_rep} --folds=${folds} --rep_start=${rep_start} --rep_end=${rep_end} --fold_start=${fold_start} --fold_end=${fold_end}
done


#### SCZ EUR

In [ ]:
method=pt2 # pt2 pt3 ls 
base_rep=105
folds=5
rep_start=1
rep_end=6
fold_start=1
fold_end=5

discoveries="PGC2_noAJ" 

imps="impute2_1kg_eur2 impute2_1kg_eas2 impute2_1kg_afr2" 
targets="dbg-scz19" 

for imp in $imps; do 
    bash run_UKB_folds_${method}.sh --discoveries=${discoveries} --targets=${targets} --imps=${imp} --ld=eur \
                                    --base_rep=${base_rep} --folds=${folds} --rep_start=${rep_start} --rep_end=${rep_end} --fold_start=${fold_start} --fold_end=${fold_end} & 
done


In [ ]:
method=pt2 # ls pt2 pt3
base_rep=105
folds=5
rep_start=1
rep_end=6
fold_start=1
fold_end=5

discoveries="PGC2_noAJ" 

imps="impute2_1kg_eur impute2_1kg_eas impute2_1kg_afr" 
targets="gain_afr" 

for imp in $imps; do 
    bash run_UKB_folds_${method}.sh --discoveries=${discoveries} --targets=${targets} --imps=${imp} --ld=eur \
                                    --base_rep=${base_rep} --folds=${folds} --rep_start=${rep_start} --rep_end=${rep_end} --fold_start=${fold_start} --fold_end=${fold_end} & 
done


#### Evaluate PRS quality using EAS GWASs

In [ ]:
method=ls # pt3 pt2
base_rep=105
folds=5
rep_start=1
rep_end=6
fold_start=1
fold_end=5

phenos=ctrt # "ctrt chol hfvr hyty ht osar madd t2d ast utfi gerx angna" 

targets="ukbb_sas" # "ukbb_sas,ukbb_afr"
imps="original,impute2_1kg_eur,impute2_1kg_afr,impute2_1kg_sas" 

for pheno in $phenos; do 
    discoveries="GC_${pheno}_sakaue_2021"
    bash run_UKB_folds_${method}.sh --discoveries=${discoveries} --targets=${targets} --imps=${imps} --pheno=${pheno} --ld=eas \
                                    --base_rep=${base_rep} --folds=${folds} --rep_start=${rep_start} --rep_end=${rep_end} --fold_start=${fold_start} --fold_end=${fold_end}  
done

#### SCZ EAS - P+T

In [ ]:
method=pt3 # pt2 pt3
base_rep=105
folds=5
rep_start=1
rep_end=6
fold_start=1
fold_end=5

discoveries="LH_PGC-SCZ-EAS" 

imps="impute2_1kg_afr" # "impute2_1kg_eur impute2_1kg_eas impute2_1kg_afr" 
targets="gain_afr" 

for imp in $imps; do 
    bash run_UKB_folds_${method}.sh --discoveries=${discoveries} --targets=${targets} --imps=${imp} --ld=eas \
                                    --base_rep=${base_rep} --folds=${folds} --rep_start=${rep_start} --rep_end=${rep_end} --fold_start=${fold_start} --fold_end=${fold_end} --override=true # & 
done


In [ ]:
method=pt3 # pt2 pt3
base_rep=105
folds=5
rep_start=1
rep_end=6
fold_start=1
fold_end=5

discoveries="LH_PGC-SCZ-EAS" 

imps="impute2_1kg_eur2 impute2_1kg_eas2 impute2_1kg_afr2" 
targets="dbg-scz19" 

for imp in $imps; do 
    bash run_UKB_folds_${method}.sh --discoveries=${discoveries} --targets=${targets} --imps=${imp} --ld=eas \
                                    --base_rep=${base_rep} --folds=${folds} --rep_start=${rep_start} --rep_end=${rep_end} --fold_start=${fold_start} --fold_end=${fold_end} --override=true # & 
done


#### SCZ EAS - Lassosum

In [ ]:
method=ls
base_rep=105
folds=5
rep_start=1
rep_end=6
fold_start=1
fold_end=5

discoveries="LH_PGC-SCZ-EAS_LS" 

imps="impute2_1kg_eur2 impute2_1kg_eas2 impute2_1kg_afr2" 
targets="dbg-scz19" 

for imp in $imps; do 
    bash run_UKB_folds_${method}.sh --discoveries=${discoveries} --targets=${targets} --imps=${imp} --ld=eas \
                                    --base_rep=${base_rep} --folds=${folds} --rep_start=${rep_start} --rep_end=${rep_end} --fold_start=${fold_start} --fold_end=${fold_end} & 
done


In [ ]:
method=ls
base_rep=105
folds=5
rep_start=1
rep_end=6
fold_start=1
fold_end=5

discoveries="LH_PGC-SCZ-EAS_LS" 

imps="impute2_1kg_afr" # "impute2_1kg_eur impute2_1kg_eas impute2_1kg_afr" 
targets="gain_afr" 

for imp in $imps; do 
    bash run_UKB_folds_${method}.sh --discoveries=${discoveries} --targets=${targets} --imps=${imp} --ld=eas \
                                    --base_rep=${base_rep} --folds=${folds} --rep_start=${rep_start} --rep_end=${rep_end} --fold_start=${fold_start} --fold_end=${fold_end} & 
done
